# Bonus: Selective ControlNet on 2 frames with single prompt

## Setup

In [ ]:
# Clone both repos
!git clone https://github.com/Weifeng-Chen/control-a-video.git
!git clone https://github.com/Reouth/SelectiveControlNet.git

# Install dependencies
!pip install diffusers==0.14.0
!pip install transformers==4.27.3
!pip install accelerate==0.21.0
!pip install xformers==0.0.16
!pip install imageio==2.33
!pip install decord==0.6.0
!pip install opencv-python
!pip install einops==0.6.0
!pip install -q "huggingface_hub==0.14.1"

## Imports

In [ ]:
import sys
sys.path.append("/content/SelectiveControlNet")
sys.path.append("/content/control-a-video")

import os
import cv2
import numpy as np
from PIL import Image
from torchvision.transforms import ToTensor
from google.colab import files


from SelectiveControlNet.src.video_utils import (
    extract_two_frames,
    frames_to_video,
    find_latest_output_video,
    save_video_frames
)

from SelectiveControlNet.src.masking import get_foreground_images

##Preprocessing
Start by loading the input video, extract 2 frames, generate masks, and create video format (mp4) for input.


In [ ]:
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

frames = extract_two_frames(video_path, [10, 20])  # 2 key frames
fg_masked = get_foreground_images(frames)
frames_to_video(fg_masked, "two_frames_video.mp4", fps=2)


Saving 3402765-uhd_4096_2160_25fps.mp4 to 3402765-uhd_4096_2160_25fps (1).mp4


## Run Control-a-video
[github repo](https://github.com/Weifeng-Chen/control-a-video.git)

In [ ]:
!python3 inference.py \
  --prompt "mermaid underwater, blue clothes, corals and fishes background"\
  --input_video two_frames_video.mp4 \
  --control_mode canny \
  --num_sample_frames 2 \
  --each_sample_frame 2 \
  --sampling_rate 1 \
  --video_scale 1.0 \
  --init_noise_thres 0.3 \
  --inference_step 50 \
  --guidance_scale 7.5 \
  --height 512 \
  --width 512

## Convert back to image format

In [ ]:
video_path = find_latest_output_video("outputs")  # or manually: "outputs/sample_0/video.mp4"

#Extract frames and save them as images
gen_frames_paths = save_video_frames(video_path, frame_indices=[0, 1], output_dir="generated_frames", prefix="frame", fmt="png")

#Display extracted frames
from PIL import Image
from IPython.display import display

for path in gen_frames_paths:
    display(Image.open(path))